# Active Fine-tuning of PLMs

## Objectives
- Trial three approaches for active fine-tuning of ESM2 on our Pikh1 HMA data
    1. train an ensemble of models and use the mean and variance of their predictions to guide learning
    2. use dropout layer during prediction, find mean and variance of predictions
    3. mean variance estimation fine-tuning to predict two values with gaussian negative log-likelihood loss
- Compare data efficiency between each method (that is, how many training labels are needed to achieve the same spearman r on a universal test set).

## Prepare data splits
Starting data is from FACS of surface displayed Pikh1 HMA variants when exposed to 1 uM AVR-PikC. There are 3960 labels in this dataset, each with a full sequence and an enrichment score, roughly correlated with binding affinity. We'll load these into a torch dataset, then split into 80% train, 10% val, and 10% test. The training data will be what we use for all active learning loops, validated on the val set. The test set will be used as a universal final test set for all models.

In [1]:
import pandas as pd

df = pd.read_csv('avrpikC_full.csv')

df.head()

,aa_sequence,enrichment_score
0,GLKQKIVIKVAMEGNNCRSKAMALVASTGGVDSVALVGDLRDKIEV...,1.468796
1,GLKRIIVIKVAREGNNCRSKAMALVASTGGVDSVALVGDLRGKIEV...,1.415944
2,GLKRIIVIKVAMEGNNCRSKAMALVASTGGVDSVALVGDLRGKIEV...,1.389615
3,GLKQKIVIKVAMEGNNCRSKAMALVASTGGVDSVALVGDLRDKIEV...,1.359651
4,GLKQKIVIKVAMEGNNCRSKAMALVASTGGVDSVALVGDLRGKIEV...,1.343857


In [2]:
sequences = df.aa_sequence
scores = df.enrichment_score

In [3]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer

class BindingDataset(Dataset):
    def __init__(self, sequences, scores):
        # make sure sequence and scores have the same length
        assert len(sequences) == len(scores), f"Sequences and scores must be of the same length.\nNumber of sequences: {len(sequences)}\nNumber of scores: {len(scores)}"
        self.sequences = sequences
        self.scores = scores
        self.tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t12_35M_UR50D")

    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self,idx):
        sequence = self.sequences[idx]
        label = torch.tensor(self.scores[idx], dtype=torch.float)

        # tokenize the sequence
        tokenized = self.tokenizer(
            sequence,
            max_length=80, # 78 residues + 2 extra tokens
            return_tensors='pt'
        )

        # return input_ids: attention masks, removing the batch dimension
        inputs = {key: val.squeeze(0) for key, val in tokenized.items()}

        return inputs, label

In [4]:
from torch.utils.data import random_split, DataLoader
torch.manual_seed(42)
torch.cuda.manual_seed(42)

device = "cuda" if torch.cuda.is_available() else 'cpu'

BATCH_SIZE = 12

full_dataset = BindingDataset(sequences, scores)

# split the data into train, val, and test sets
train_size = int(0.8 * len(full_dataset))
val_size = int(0.1 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size
training_pool, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

# define dataloaders for val and test sets, train will be defined later for subsets
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

## Approach 1: Ensemble Predictions
1. Initialize model 1: pretrained esm2 with a randomized binding head
2. Train on an initial batch of 24 samples (test random and zero-shot predictions).
    - Initial hyperparameters (note that during an actual active learning run, you can't change this in the middle of the campaign)
        - optimizer: AdamW
        - learning rate: 2e-5
        - weight decay: 0.01
        - early stopping
3. Evaluate on validation set
4. Evalutate on the rest of the available training pool
5. Initialize model 2
6. Repeat 2-5 until 5 models have been trained and used to evalutate
7. Calculate variance of model predictions on each sequence in the training pool.
8. Select the next 24 sequences with the highest variance.
9. Reinitialize model 1 with pretrained esm2 and a freshly randomized binding head.
10. Repeat ensemble training, evaluation, and acquisition

- Notes about alternative approaches for the baseline fine-tuning approach:
    - Fine-tuning ESM2 is still under active research, and recent literature suggests that just using the CLS token for mutant effects may not be the best approach
    - [ESM Effect](https://www.biorxiv.org/content/10.1101/2025.02.03.635741v1.full.pdf) details a framework for using the mutant token vectors as the basis for predicting mutant effects.
        - Here, they use 35M model with last two layers unfrozen. They use the mutant tokens to predict mutant effects. They have a prediction head composed of two linear layers. However, their datasets are of deep mutational scans, so only one mutation. I would have to adapt this for higher numbers of mutations.
            - to adapt it for a higher number of mutations, I could pool, concatenate, or use an attention mechanism
- For now, I'm going to set up the active training loop using the standard CLS token approach (aka, what the AutoModelForSequenceClassification class does from the transformers library), but I will need to separately experiment with other approaches.

In [5]:
from transformers import AutoModelForSequenceClassification
from torch.nn import MSELoss
from torch.optim import AdamW
from tqdm import tqdm
from torchmetrics.regression import SpearmanCorrCoef

MODEL_NAME = "facebook/esm2_t6_8M_UR50D"
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 0.01
ES_PATIENCE = 8

def initialize_HF_ESM2():
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels = 1)
    # loss_fn = MSELoss() : HuggingFace automatically handles the loss
    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
    spearman = SpearmanCorrCoef()
    return model, optimizer, spearman #,loss_fn

def train_step(model, optimizer, train_dataloader):
    model.to(device)
    model.train()
    total_train_loss = 0
    for inputs, labels in tqdm(train_dataloader, desc=f"[Training]"):
        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(**inputs, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
    
    avg_train_loss = total_train_loss / len(train_dataloader)
    return avg_train_loss

def val_step(model, val_dataloader, spearman):
    model.eval()
    total_val_loss = 0

    all_preds = []
    all_labels = []

    with torch.inference_mode():
        for inputs, labels in tqdm(val_dataloader, desc=f"[Validation]"):
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            outputs = model(**inputs, labels=labels)
            preds = outputs.logits.squeeze() # to make sure dimensions are the same for spearman
            loss = outputs.loss

            total_val_loss += loss.item()

            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())

    avg_val_loss = total_val_loss / len(val_dataloader)

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    spearmanr = spearman(all_preds, all_labels).item()

    return avg_val_loss, spearmanr

In [6]:
def initialize_and_train_new_model(epochs, train_dataloader, val_dataloader):
    model, optimizer, spearman = initialize_HF_ESM2()
    for epoch in range(epochs):
        print(f'\nEpoch {epoch + 1}/{epochs}\n----------------------------')
        train_loss = train_step(model, optimizer, train_dataloader)
        val_loss, spearmanr = val_step(model, val_dataloader, spearman)
        print(f'Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | SpearmanR: {spearmanr:.4f}')
    return model

In [7]:
import numpy as np
from torch.utils.data import Subset

def acquire_new_batch(dataset, n_samples_to_acquire, labeled_indices, unlabeled_indices, acquisition_scores=None):
    # if initial batch, when there are no acquisition scores, select randomly
    if acquisition_scores is None:
        n_samples_to_acquire = min(n_samples_to_acquire, len(unlabeled_indices))
        indices_to_acquire = np.random.choice(unlabeled_indices, size=n_samples_to_acquire, replace=False)
    # else select based on top acquisition scores
    else:
        # make sure we don't overshoot samples to acquire if on the final batch
        n_samples_to_acquire = min(n_samples_to_acquire, len(acquisition_scores))
        # get the indicies of the top acquisition scores (num of samples)
        top_k_indices = acquisition_scores.topk(n_samples_to_acquire).indices
        # use these to find the indicies that map back to the original dataset
        indices_to_acquire = unlabeled_indices[top_k_indices.cpu().numpy()]
    
    # update the indices lists
    labeled_indices = np.concatenate([labeled_indices, indices_to_acquire])
    unlabeled_indices = np.setdiff1d(unlabeled_indices, indices_to_acquire, assume_unique=True)
    
    # create new subsets and dataloaders
    train_subset = Subset(dataset, labeled_indices.tolist())
    pool_subset = Subset(dataset, unlabeled_indices.tolist())
    train_dataloader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
    pool_dataloader = DataLoader(pool_subset, batch_size=BATCH_SIZE, shuffle=False)
    
    return train_dataloader, pool_dataloader, labeled_indices, unlabeled_indices

In [8]:
# get model predictions on unlabeled dataset given model and pool dataloader
def get_model_predictions(model, pool_dataloader):
    # set the model to eval mode
    model.eval()
    # define list to store predictions
    all_preds = []
    # set inference mode
    with torch.inference_mode():
        # iterate through pool loader
        for inputs, labels in tqdm(pool_dataloader, desc=f"[Surveying]"):
            # get model predictions, append them to list (num batches, batch size)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = model(**inputs, labels=labels)
            preds = outputs.logits
            all_preds.append(preds.cpu())
    # concat predictions from all batches for a single prediction tensor
    all_preds = torch.cat(all_preds)
    # return full prediction tensor
    return all_preds

In [9]:
EPOCHS = 20
# train ensemble given training dataloader, pool dataloader, validation set, and number of models
def train_ensemble(n_models, train_dataloader, pool_dataloader, val_dataloader):
    # define list to store ensemble predictions
    ensemble_predictions = []
    # for i in number of models
    for i in range(n_models):
        print(f"Training Model {i+1}...")
        # set manual seed i
        torch.manual_seed(i)
        torch.cuda.manual_seed(i)
        # initialize and train
        model = initialize_and_train_new_model(EPOCHS, train_dataloader, val_dataloader)
        # get model predictions on pool dataloader, append to list
        print(f"Using Model {i+1} to evaluate remaining training pool...")
        pool_preds = get_model_predictions(model, pool_dataloader)
        ensemble_predictions.append(pool_preds)
    # stack ensemble predictions to create tensor of shape (n_models, n_unlabeled_samples)
    ensemble_predictions = torch.stack(ensemble_predictions, dim=0)
    print("Ensemble training complete, submitting predictions for next cycle.")
    # return list of ensemble predictions
    return ensemble_predictions

In [10]:
# get acquisition scores (variance) given model predictions
def get_acquisition_scores(ensemble_predictions):
    # calculate variance for each index
    variances = torch.var(ensemble_predictions, dim=0)
    # return list of acquisition scores
    return variances.squeeze()

In [11]:
NUM_MODELS = 5

# run active learning campaign given num samples, num samples per batch
def run_active_learning_campaign(n_samples, n_samples_per_batch, training_pool, val_dataloader):
    # initialize index lists
    total_pool_size = len(training_pool)
    unlabeled_indices = np.arange(total_pool_size)
    labeled_indices = np.array([], dtype=np.int64)

    ensemble_predictions = None
    
    while len(labeled_indices) < n_samples and len(unlabeled_indices) > 0:
        current_cycle = (len(labeled_indices) // n_samples_per_batch) + 1
        print(f"[Cycle {current_cycle}]\n")

        # Pass the current index lists to the unified function
        if ensemble_predictions is None:
            print(f"Choosing initial {n_samples_per_batch} samples randomly...")
            train_dataloader, pool_dataloader, labeled_indices, unlabeled_indices = acquire_new_batch(
                training_pool, n_samples_per_batch, labeled_indices, unlabeled_indices, acquisition_scores=None
            )
        else:
            scores = get_acquisition_scores(ensemble_predictions)
            print(f"Selecting new data points...")
            train_dataloader, pool_dataloader, labeled_indices, unlabeled_indices = acquire_new_batch(
                training_pool, n_samples_per_batch, labeled_indices, unlabeled_indices, acquisition_scores=scores
            )
        
        # Stop if the unlabeled pool is now empty
        if len(unlabeled_indices) == 0:
            print("Unlabeled pool is empty. Proceeding to final model training.")
            break

        print("Starting ensemble training and pool evaluation...")
        ensemble_predictions = train_ensemble(NUM_MODELS, train_dataloader, pool_dataloader, val_dataloader)

    print("\nActive learning campaign complete.")
    print(f"Training final model on {len(labeled_indices)} actively selected samples...")
    model = initialize_and_train_new_model(EPOCHS, train_dataloader, val_dataloader)
    
    return model

In [12]:
model_72_active = run_active_learning_campaign(72, 24, training_pool, val_dataloader)

Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Cycle 1]

Choosing initial 24 samples randomly...
Starting ensemble training and pool evaluation...
Training Model 1...

Epoch 1/20
----------------------------


/home/oscar/miniconda3/envs/ml/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
[Validation]: 100%|██████████| 33/33 [00:00<00:00, 151.88it/s]


Train Loss: 0.3040 | Val Loss: 0.2015 | SpearmanR: -0.2227

Epoch 2/20
----------------------------


[Validation]: 100%|██████████| 33/33 [00:00<00:00, 146.63it/s]


...

Train Loss: 0.1613 | Val Loss: 0.3058 | SpearmanR: 0.3157

Epoch 19/20
----------------------------


[Validation]: 100%|██████████| 33/33 [00:00<00:00, 153.32it/s]


Train Loss: 0.1183 | Val Loss: 0.1690 | SpearmanR: 0.3754

Epoch 20/20
----------------------------


[Validation]: 100%|██████████| 33/33 [00:00<00:00, 148.33it/s]

Train Loss: 0.1032 | Val Loss: 0.2170 | SpearmanR: 0.3755


In [13]:
# run standard fine-tuning procedure given num samples
def run_standard_finetuning(n_samples, training_pool, val_dataloader):
    # get dataloader of random train data
    random_indices = torch.randperm(len(training_pool))[:n_samples].tolist()
    train_subset = Subset(training_pool, random_indices)
    train_dataloader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)

    # train model
    model = initialize_and_train_new_model(EPOCHS, train_dataloader, val_dataloader)
    return model
    # return model

In [14]:
model_72_standard = run_standard_finetuning(72, training_pool, val_dataloader)

Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1/20
----------------------------


[Validation]: 100%|██████████| 33/33 [00:00<00:00, 149.22it/s]


Train Loss: 0.1580 | Val Loss: 0.1989 | SpearmanR: -0.0337

Epoch 2/20
----------------------------


[Validation]: 100%|██████████| 33/33 [00:00<00:00, 147.03it/s]


...

Train Loss: 0.0815 | Val Loss: 0.1669 | SpearmanR: 0.4476

Epoch 19/20
----------------------------


[Validation]: 100%|██████████| 33/33 [00:00<00:00, 110.73it/s]


Train Loss: 0.0618 | Val Loss: 0.1929 | SpearmanR: 0.4501

Epoch 20/20
----------------------------


[Validation]: 100%|██████████| 33/33 [00:00<00:00, 91.41it/s]

Train Loss: 0.0800 | Val Loss: 0.1914 | SpearmanR: 0.4595


In [15]:
import torch
from tqdm import tqdm
from torchmetrics.regression import SpearmanCorrCoef, PearsonCorrCoef, MeanSquaredError

def test_model(model, test_dataloader):
    # Initialize metrics
    spearman = SpearmanCorrCoef().to(device)
    pearson = PearsonCorrCoef().to(device)
    mse = MeanSquaredError().to(device)

    model.to(device)
    model.eval()
    
    total_test_loss = 0
    all_preds = []
    all_labels = []

    with torch.inference_mode():
        for inputs, labels in tqdm(test_dataloader, desc="[Testing]"):
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            outputs = model(**inputs, labels=labels)
            preds = outputs.logits.squeeze()
            loss = outputs.loss

            total_test_loss += loss.item()

            all_preds.append(preds)
            all_labels.append(labels)

    # Concatenate all predictions and labels from all batches
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    # Calculate final metrics
    avg_test_loss = total_test_loss / len(test_dataloader)
    spearman_r = spearman(all_preds, all_labels).item()
    pearson_r = pearson(all_preds, all_labels).item()
    final_mse = mse(all_preds, all_labels).item()

    # Print the final report
    print("\n--- Final Test Report ---")
    print(f"  Test Loss: {avg_test_loss:.4f}")
    print(f"  Spearman's Rho: {spearman_r:.4f}")
    print(f"  Pearson's Rho: {pearson_r:.4f}")
    print(f"  Mean Squared Error (MSE): {final_mse:.4f}")
    print("-------------------------\n")

In [16]:
test_model(model_72_active, test_dataloader)

/home/oscar/miniconda3/envs/ml/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
[Testing]: 100%|██████████| 33/33 [00:00<00:00, 70.98it/s]



--- Final Test Report ---
  Test Loss: 0.1918
  Spearman's Rho: 0.4013
  Pearson's Rho: 0.4261
  Mean Squared Error (MSE): 0.1918
-------------------------



In [17]:
test_model(model_72_standard, test_dataloader)

/home/oscar/miniconda3/envs/ml/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
[Testing]: 100%|██████████| 33/33 [00:00<00:00, 59.05it/s]


--- Final Test Report ---
  Test Loss: 0.1799
  Spearman's Rho: 0.4412
  Pearson's Rho: 0.4200
  Mean Squared Error (MSE): 0.1799
-------------------------



- More notes
    - This simulation plays out in a defined pool of samples, how will this work in the real world, where there's practically infinite options?
        - first, you need to define the limits of the space. probably something like a limited number of mutations per sequence, and perhaps a distribution of those mutation numbers in your final set
        - boring approach: generate mutants and evaluate ensemble on mutants for a defined amount of time, select the highest variance mutants from that set.
        - cool approach: train an adversarial model that learns how to optimally challenge the plm model